In [0]:
%sql
use catalog setorfarmaceutico;
use schema gold;

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fct_estoque_principio_ativo AS
SELECT
    nome_principio_ativo,
    fabricante,
    SUM(`quantidade_disponível`) AS estoque_total,
    ROUND(MIN(`quantidade_disponível`), 2) AS estoque_min,
    ROUND(MAX(`quantidade_disponível`), 2) AS estoque_max,
    ROUND(MIN(`valor_unitário`), 2) AS preco_min,
    ROUND(MAX(`valor_unitário`), 2) AS preco_max
FROM silver.stg_principios_ativos
WHERE LOWER(status) = 'disponível'
GROUP BY 1,2
ORDER BY nome_principio_ativo, fabricante;


In [0]:
%sql
select * from fct_estoque_principio_ativo

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fct_producao_lote AS
SELECT
    nome_produto,
    quantidade_fabricada,
    data_fabricacao,
    data_validade,
    DATEDIFF(data_validade, data_fabricacao) AS validade_dias,
    CASE
        WHEN current_date() > data_validade                  THEN 'Vencido'
        WHEN DATEDIFF(data_validade, current_date()) <= 30   THEN 'Vence em até 30 dias'
        WHEN DATEDIFF(data_validade, current_date()) <= 90   THEN 'Vence em 31–90 dias'
        ELSE 'Válido (> 90 dias)'
    END AS status_validade,
    lote_fabricacao,
    responsavel_lote,
    fabrica,
    local_armazenamento
FROM silver.stg_produtos_fabricados;


In [0]:
%sql
select * from fct_producao_lote;

In [0]:
%sql
CREATE OR REPLACE TABLE gold.fct_vendas_resumo AS
SELECT
 
  date_format(date_trunc('month', data_emissao_nf), 'yyyy-MM') AS mes,
  estado,
  tipo_cliente,
  nome_produto,


  ROUND(SUM(quantidade_vendida), 2)                  AS qtd_vendida,
  ROUND(SUM(valor_total), 2)                         AS faturamento_bruto,
  ROUND(SUM(imposto_reais), 2)                       AS impostos_totais,
  ROUND(SUM(valor_total - imposto_reais), 2)         AS receita_liquida,

  ROUND(AVG(valor_total / quantidade_vendida), 2)    AS preco_medio_unitario,
  ROUND(AVG(DATEDIFF(data_vencimento, data_emissao_nf)), 0) AS prazo_medio_dias,
  ROUND(AVG((imposto_reais / valor_total) * 100), 2) AS imposto_efetivo_medio_pct

FROM silver.stg_vendas_produtos_finais
GROUP BY
  date_format(date_trunc('month', data_emissao_nf), 'yyyy-MM'),
  estado,
  tipo_cliente,
  nome_produto;


In [0]:
%sql
select * from fct_vendas_resumo;